# 트리의 앙상블

## 랜덤포레스트
정형 데이터를 다루는 데에 가장 뛰어난 성과를 내는 알고리즘을 앙상블 학습이라고 합니다.  
랜덤 포레스트는 앙상블 학습 중 하나로, 안정적인 성능을 보여줍니다.  
랜덤 포레스트에서는 결정트리를 랜덤하게 만들어 다수의 결정트리를 모은 후 분류 또는 평균 예측치를 출력합니다.  
샘플 데이터에서 랜덤하게 추출하여 훈련데이터를 만드는데, 이렇게 만들어진 샘플을 부트스트랩 샘플이라고 하고 훈련세트의 크기와 같게 만들어집니다.  
<br/>
RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용해보겠습니다. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

cross_validate를 사용하여 교차검증을 수행합니다.

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


앞의 랜덤 포레스트 모델을 훈련세트에 fit 한 후 특성 중요도를 출력해봅니다.

In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


랜덤 포레스트는 자체적으로 점수를 매기는 기능이 있습니다. 부트스트랩에 사용되지 않은 나머지 데이터를 사용하여 평가를 내릴 수 있습니다. 이를 oob 샘플이라고 합니다. 이를 사용하기 위해서는 모델에 oob_score=True를 지정해주면 됩니다.

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


oob 샘플을 사용함으로써 훈련세트에 더 많은 샘플을 사용할 수 있습니다.

## 엑스트라트리

엑스트라 트리와 랜덤 포레스트의 차이점은 엑스트라트리에서는 부트스트랩을 사용하지 않는다는 것입니다. 결정트리를 만들 때 전체 훈련 세트를 사용합니다. 다만 노드를 분할할 때 무작위로 분할합니다. 성능이 낮아질 수 있으나 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증세트의 점수를 높이는 효과가 있습니다.  
ExtraTreesClassifier을 사용하여 교차검증 점수를 확인합니다.

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


보통 엑스트라트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정트리를 훈련해야 하지만, 랜덤하게 분할하기 때문에 계산속도가 빠르다는 것이 장점입니다. 결정트리에서 최적의 분할을 찾는데에 많은 시간을 소요하기 때문에 무작위로 나눌 시에 훨씬 빨리 트리를 구성할 수 있습니다.  
엑스트라 트리에서도 특성 중요도를 출력해보겠습니다.

In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

그레이디언트 부스팅은 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블한다고 합니다.  
깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있습니다.  

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대적합이 거의 되지 않은 것을 확인할 수 있습니다. 학습률을 증가시키고 트리의 갯수를 늘려보겠습니다. 
트리의 갯수는 n_estimators, 학습률은 learning_rate로 지정할 수 있습니다.

In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


그레이디언트 부스팅에서도 특성 중요도를 알아봅니다.

In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 성능이 더 좋지만, 속도가 느리다는 단점이 있습니다.  
그레이디언트 부스팅의 속도와 성능을 개선한 것이 히스토그램 기반 그레이디언트 부스팅입니다.

## 히스토그램 기반 그레이디언트 부스팅

히스토그램 기반 그레이디언트 부스팅은 입력 특성을 256 구간으로 나눈 후 노드를 분할할 때 최적을 분할을 빨리 찾을 수 있는 것입니다.  

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 억제하는 장점을 잘 지키면서 더 높은 성능이 나옴을 알 수 있습니다.  
훈련 세트에서의 특성 중요도를 계산해보겠습니다. 

In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


이제 테스트세트에서의 중요도를 계산해보겠습니다.

In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


그리고 테스트 세트에서의 성능을 확인합니다.

In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 다른 라이브러리인 XGBoost를 사용해서도 구현해보겠습니다.

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677
